# Tutorial - Keras implementation of Deblur GAN with Demonstration.

# Introduction

__Generative Adversarial Networks (GAN)__ is one of the most promising recent developments in Deep Learning. GAN, introduced by Ian Goodfellow in 2014, attacks the problem of unsupervised learning by training two deep networks, called _Generator_ and _Discriminator_, that compete and cooperate with each other. In the course of training, both networks eventually learn how to perform their tasks.

Here is a brief of the research paper on DeblurGAN. The paper can be downloaded from [here](https://arxiv.org/pdf/1711.07064.pdf "Deblur GAN")


![alt text](https://cdn-images-1.medium.com/max/800/1*N4oqJsGmH-KZg3Vqrm_uYw.jpeg)

To get a nice intuition consider it this way. 

Frank Abagale is a counterfiet artist. He knows how to make fake cheques. (Yup! That's from the movie - Catch me if You Can).

Frank makes a fake cheque and show it to FBI. The FBI identifies it as a fake and tell Frank how they identified that. Our hero now improvises with the new little info and creates another fake. The FBI again identifies it's fakeness and tells Frank how they identified it. 

*This process of improving and providing feedback keeps going on untill Frank successfully fools the FBI by making a perfect counterfeit!*

**Let's take each block one by one.**



## What is Blind Motion Deblurring and why we use GAN's for that?
 
In simple words, removing the blurring effect caused due to motion of objects (or camera ) in images is called Motion Deblurring. The common formulation of non-uniform blur model is the following:


  __IB = k(M) * IS + N__ where

* __IB__ is a blurred image, 

* __k(M)__ are unknown blur kernels determined by motion field

* operator __(*)__  denotes the convolution

* __IS__ is the sharp latent

* __N__ is an additive noise.


If __k(M)__ is unknown the problem is classified as __Blind Deblurring__.

We use GANs as they are known for the ability to preserve texture details in images, create solutions that are close to the real image manifold and look perceptually convincing.

We are using GoPro dataset that consists of 2103 pairs of blurred and sharp images in 720p quality, taken from various scenes.


# The Generator Model

The generator aims at reproducing sharp images. The network is based on ResNet blocks. It keeps track of the evolutions applied to the original blurred image

The generator synthesizes fake images. In Figure 2, the fake image is generated from a 100-dimensional noise (uniform distribution between -1.0 to 1.0) using the inverse of convolution, called transposed convolution. Instead of fractionally-strided convolution as suggested in DCGAN, upsampling between the first three layers is used since it synthesizes more realistic handwriting images. In between layers, batch normalization stabilizes learning. The activation function after each layer is a ReLU. The output of the sigmoid at the last layer produces the fake image. Dropout of between 0.3 and 0.5 at the first layer prevents overfitting. Listing 2 shows the implementation in Keras.

<img src="https://image.ibb.co/bMXkFT/image.png" alt="image" border="0">



The core is 9 ResNet blocks applied to an upsampling of the original image. A ResNet block is defined as shown below.

![alt text](https://cdn-images-1.medium.com/max/1000/1*OhuvC1YUdHyLbGO6rWWHhA.png)

First we take the input and pad it using the utility function ReflectionPadding2D defined in the layer_utils.py




In [0]:
def res_block(input, filters, kernel_size=(3, 3), strides=(1, 1), use_dropout=False):
    """
    Instanciate a Keras Resnet Block using sequential API.
    :param input: Input tensor
    :param filters: Number of filters to use
    :param kernel_size: Shape of the kernel for the convolution
    :param strides: Shape of the strides for the convolution
    :param use_dropout: Boolean value to determine the use of dropout
    :return: Keras Model
    """
    x = ReflectionPadding2D((1, 1))(input)
    x = Conv2D(filters=filters,
               kernel_size=kernel_size,
               strides=strides,)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    if use_dropout:
        x = Dropout(0.5)(x)

    x = ReflectionPadding2D((1, 1))(x)
    x = Conv2D(filters=filters,
               kernel_size=kernel_size,
               strides=strides,)(x)
    x = BatchNormalization()(x)

    merged = Add()([input, x])
    return merged

In D-GAN we create 9 ResNet blocks layer and pass the unsampled version of the input. To keep the output normalized 

In [0]:
from keras.layers import Input, Activation, Add
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.core import Lambda
from keras.layers.normalization import BatchNormalization
from keras.models import Model

from layer_utils import ReflectionPadding2D, res_block

ngf = 64
input_nc = 3
output_nc = 3
input_shape_generator = (256, 256, input_nc)
n_blocks_gen = 9


def generator_model():
    """Build generator architecture."""
    # Current version : ResNet block
    inputs = Input(shape=image_shape)

    x = ReflectionPadding2D((3, 3))(inputs)
    x = Conv2D(filters=ngf, kernel_size=(7,7), padding='valid')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Increase filter number
    n_downsampling = 2
    for i in range(n_downsampling):
        mult = 2**i
        x = Conv2D(filters=ngf*mult*2, kernel_size=(3,3), strides=2, padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

    # Apply 9 ResNet blocks
    mult = 2**n_downsampling
    for i in range(n_blocks_gen):
        x = res_block(x, ngf*mult, use_dropout=True)

    # Decrease filter number to 3 (RGB)
    for i in range(n_downsampling):
        mult = 2**(n_downsampling - i)
        x = Conv2DTranspose(filters=int(ngf * mult / 2), kernel_size=(3,3), strides=2, padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

    x = ReflectionPadding2D((3,3))(x)
    x = Conv2D(filters=output_nc, kernel_size=(7,7), padding='valid')(x)
    x = Activation('tanh')(x)

    # Add direct connection from input to output and recenter to [-1, 1]
    outputs = Add()([x, inputs])
    outputs = Lambda(lambda z: z/2)(outputs)

    model = Model(inputs=inputs, outputs=outputs, name='Generator')
    return model

# The Discriminator Model



A discriminator that tells how real an image is, is basically a deep Convolutional Neural Network (CNN) as shown in Figure.

The sigmoid output is a scalar value of the probability of how real the image is (0.0 is certainly fake, 1.0 is certainly real, anything in between is a gray area). 

The difference from a typical CNN is the absence of max-pooling in between layers. Instead, a strided convolution is used for downsampling.

The activation function used in each CNN layer is a leaky ReLU. A dropout between 0.4 and 0.7 between layers prevent over fitting and memorization. 

<img src="https://image.ibb.co/gdJeaT/image.png" alt="image" border="0">

Here is the architecture for Discriminator.

In [0]:
from keras.layers import Input
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D
from keras.layers.core import Dense, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Model

ndf = 64
output_nc = 3
input_shape_discriminator = (256, 256, output_nc)


def discriminator_model():
    """Build discriminator architecture."""
    n_layers, use_sigmoid = 3, False
    inputs = Input(shape=input_shape_discriminator)

    x = Conv2D(filters=ndf, kernel_size=(4,4), strides=2, padding='same')(inputs)
    x = LeakyReLU(0.2)(x)

    nf_mult, nf_mult_prev = 1, 1
    for n in range(n_layers):
        nf_mult_prev, nf_mult = nf_mult, min(2**n, 8)
        x = Conv2D(filters=ndf*nf_mult, kernel_size=(4,4), strides=2, padding='same')(x)
        x = BatchNormalization()(x)
        x = LeakyReLU(0.2)(x)

    nf_mult_prev, nf_mult = nf_mult, min(2**n_layers, 8)
    x = Conv2D(filters=ndf*nf_mult, kernel_size=(4,4), strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)

    x = Conv2D(filters=1, kernel_size=(4,4), strides=1, padding='same')(x)
    if use_sigmoid:
        x = Activation('sigmoid')(x)

    x = Flatten()(x)
    x = Dense(1024, activation='tanh')(x)
    x = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=x, name='Discriminator')
    return model

The last step is to build the full model. A particularity of this GAN is that inputs are real images and not noise. Therefore, we have a direct feedback on the generator’s outputs.


In [0]:
from keras.layers import Input
from keras.models import Model

def generator_containing_discriminator_multiple_outputs(generator, discriminator):
    inputs = Input(shape=image_shape)
    generated_images = generator(inputs)
    outputs = discriminator(generated_images)
    model = Model(inputs=inputs, outputs=[generated_images, outputs])
    return model

# Losses

We extract losses at two levels, at the end of the generator and at the end of the full model.

The first one is a perceptual loss computed directly on the generator’s outputs. This first loss ensures the GAN model is oriented towards a deblurring task. It compares the outputs of the first convolutions of VGG.

In [0]:
import keras.backend as K
from keras.applications.vgg16 import VGG16
from keras.models import Model

image_shape = (256, 256, 3)

def perceptual_loss(y_true, y_pred):
    vgg = VGG16(include_top=False, weights='imagenet', input_shape=image_shape)
    loss_model = Model(inputs=vgg.input, outputs=vgg.get_layer('block3_conv3').output)
    loss_model.trainable = False
    return K.mean(K.square(loss_model(y_true) - loss_model(y_pred)))

The second loss is the Wasserstein loss performed on the outputs of the whole model. It takes the mean of the differences between two images. It is known to improve convergence of generative adversarial networks.

In [0]:
import keras.backend as K

def wasserstein_loss(y_true, y_pred):
    return K.mean(y_true*y_pred)

## Training the Model
The first step is to load the data and initialize all the models. We use our custom function to load the dataset, and add Adam optimizers for our models. We set the Keras trainable option to prevent the discriminator from training.



In [0]:
# Load dataset
data = load_images('./images/train', n_images)
y_train, x_train = data['B'], data['A']

# Initialize models
g = generator_model()
d = discriminator_model()
d_on_g = generator_containing_discriminator_multiple_outputs(g, d)

# Initialize optimizers
g_opt = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
d_opt = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
d_on_g_opt = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

# Compile models
d.trainable = True
d.compile(optimizer=d_opt, loss=wasserstein_loss)
d.trainable = False
loss = [perceptual_loss, wasserstein_loss]
loss_weights = [100, 1]
d_on_g.compile(optimizer=d_on_g_opt, loss=loss, loss_weights=loss_weights)
d.trainable = True

Then, we start launching the epochs and divide the dataset into batches.

In [0]:
for epoch in range(epoch_num):
  print('epoch: {}/{}'.format(epoch, epoch_num))
  print('batches: {}'.format(x_train.shape[0] / batch_size))

  # Randomize images into batches
  permutated_indexes = np.random.permutation(x_train.shape[0])

  for index in range(int(x_train.shape[0] / batch_size)):
      batch_indexes = permutated_indexes[index*batch_size:(index+1)*batch_size]
      image_blur_batch = x_train[batch_indexes]
      image_full_batch = y_train[batch_indexes]

Finally, we successively train the discriminator and the generator, based on both losses. We generate fake inputs with the generator. We train the discriminator to distinguish fake from real inputs, and we train the whole model.

In [0]:
for epoch in range(epoch_num):
  for index in range(batches):
    # [Batch Preparation]

    # Generate fake inputs
    generated_images = g.predict(x=image_blur_batch, batch_size=batch_size)
    
    # Train multiple times discriminator on real and fake inputs
    for _ in range(critic_updates):
        d_loss_real = d.train_on_batch(image_full_batch, output_true_batch)
        d_loss_fake = d.train_on_batch(generated_images, output_false_batch)
        d_loss = 0.5 * np.add(d_loss_fake, d_loss_real)

    d.trainable = False
    # Train generator only on discriminator's decision and generated images
    d_on_g_loss = d_on_g.train_on_batch(image_blur_batch, [image_full_batch, output_true_batch])

    d.trainable = True

# Demonstration

Here is the execution of deblurring an artificially blurred (street.png). 
### Note we are using pretrained models as fresh execution of the code explained above requires abundant execution time and computational resources.

In [1]:
!git clone https://github.com/mathurk29/deblur-gan-tutorial.git #A file by name deblura.png should get downloaded after running  this and following code blocks.

Cloning into 'deblur-gan-tutorial'...
remote: Counting objects: 72, done.
remote: Total 72 (delta 0), reused 0 (delta 0), pack-reused 72
Unpacking objects: 100% (72/72), done.


In [2]:
!pip install -r deblur-gan-tutorial/requirements.txt
!pip install -r deblur-gan-tutorial/requirements-gpu.txt
!pip install click

    100% |████████████████████████████████| 81kB 4.3MB/s 
    100% |████████████████████████████████| 71kB 7.0MB/s 
    100% |████████████████████████████████| 5.4MB 8.7MB/s 
    100% |████████████████████████████████| 890kB 26.0MB/s 
    100% |████████████████████████████████| 3.3MB 12.5MB/s 
    100% |████████████████████████████████| 327kB 24.0MB/s 
    100% |████████████████████████████████| 17.2MB 2.1MB/s 
    100% |████████████████████████████████| 5.9MB 8.8MB/s 
  Could not find a version that satisfies the requirement pkg-resources==0.0.0 (from -r deblur-gan-tutorial/requirements.txt (line 17)) (from versions: )


No matching distribution found for pkg-resources==0.0.0 (from -r deblur-gan-tutorial/requirements.txt (line 17))
  Using cached https://files.pythonhosted.org/packages/42/3c/1985d86a44bfe44fd060c02807336f840a509bfaa2d340860fba7d22da39/absl-py-0.1.9.tar.gz
  Using cached https://files.pythonhosted.org/packages/33/70/86c5fec937ea4964184d4d6c4f0b9551564f821e1c3575907639036d9b90/bleach-1.5.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/34/c1/8806f99713ddb993c5366c362b2f908f18269f8d792aff1abfd700775a77/click-6.7-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e1/5a/53db15bf367d2028bdc6700dbdf1bdfab46b9f208b7516952817c0808118/decorator-4.2.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/cc/26/b61e3a4eb50653e8a7339d84eeaa46d1e93b92951978873c220ae64d0733/futures-3.1.1.tar.gz
  Using cached https://files.pythonhosted.org/packages/f2/b8/a63fcc840bba5c76e453dd712dbca63178a264c8990e0086b72965d4e954/h5py-2.7.1-

In [5]:
import os
os.chdir("/content/deblur-gan-tutorial")
!python deblur_image.py --image_path a.png
from google.colab import files
files.download('deblura.png')

Using TensorFlow backend.
